## Reading the data

In this notebook, we show how to read the dataset. 

Our dataset can be found [here](https://data.world/maxstrange/diplomacyboardgame).

During our analysis, we realised that 'units' have a duplication problem. For several games, all units were replicated. Hence, we remove those. By precaution, we also remove duplicate for orders and for turns. 

In [2]:
import pandas as pd
import time
from collections import deque
import numpy as np
pd.options.mode.chained_assignment = None

In [3]:
# read the dataframes
all_games = pd.read_pickle("data/games.pkl")
all_orders = pd.read_pickle("data/orders.pkl")
all_players = pd.read_pickle("data/players.pkl")
all_turns = pd.read_pickle("data/turns.pkl")
all_units = pd.read_pickle("data/units.pkl")

# remove duplicates
all_units = all_units.drop_duplicates()

In [4]:
all_games.head(3)

,id,num_turns,num_players
0,37317,166,7
1,37604,51,7
2,39337,101,7


## Detecting betrayal

What I want to do here is **to detect betrayals within a game**, using the same definitions as in the paper we have studied. Let's recall a few things, and we will explore the dataset based on those definitions.

### What are game actions ? 

Each player has **units** (one per each city a player controls) and thoses are moved using **orders**. There are 2 kinds of orders: 
- **support** order: two units join and become bigger (i.e. stronger). One player can support another.
- movement: move a unit somewhere. If it meets another player's unit, it will be a **battle**

### How to define relationships ? 

Let's follow the definitions given by the paper.

**Act of friendship**: when a player supports another.

**Act of hostility**: When a player invades another, or if a player supports an invasion to the other player's territory.

**Friendship**: a relationship between two players spanning over 3 seasons containing at least 2 **consecutive and reciprocated** acts of friendships.

**Betrayal** / **Broken friendship**: When, after being in a friendship, two players engage in at least 2 acts of hostility. 

### Additional information required for data-processing

It is important to understand the [rules of the game](https://www.playdiplomacy.com/help.php?sub_page=Game_Rules).

Here is a list of points we want to raise before starting the programming, obtained from looking at the rules.
- Each **year** is breaked down into 2 **seasons**: {'Spring', 'Fall'}.
- Each **seasons** is itself divided into several phases, called **turns** (therefore, a year is made of at least 2 turns, and not more than 5):
    - **orders**: each player submit orders to all of its units (that can be **hold**, **move**, **support** or **convoy**)
    - **retreats**: a phase that happens when some units (called **disloged units**) need to retreat. If they can't, they are destroyed
    - **builds**: only happens after the *fall retreat*. Players gain control of SCS they are occupying.
- Geographically, the game is divided into **provinces**
- some provinces are called **supply centers** (SCS) and to win a **player** must control 18 supply centers.
- Each **unit** belongs to a **player** and there can be **only 1 unit** in a province at a time, however **units** can join their force with **support order**.
- There are 2 types of **units**:  {'F' or 'A'} for {Fleet, Army}
- Each **player** is characterized by its country, encoded by a letter: {E,F,I,G,A,T,R} standing for {England, France, Italy, Germany, Austria, Turkey, Russia}

We also give an clarification for the rows of 'all_orders' (i.e. the proper orders) because we will be using those quite a lot, and it can be hard to understand. 
- orders are defined by a **game_id**, a **unit_id** and **turn_number** (which makes sense, considering all the above points). 
- each order has a field **location** which is the province of origin of the unit
- depending on the **unit_order**, here is the description of the fields

| unit_order | location                 | target                            | target_dest     |
| ---------- | ------------------------ | --------------------------------- | --------------- |
| MOVE       | initial loc. of the unit | loc. to move to                   | null            |
| HOLD       | initial loc. of the unit | null                              | null            |
| CONVOY     |                          | initial loc.                      | end goal loc.   |
| SUPPORT    |                          | loc. of unit to be supported      | its target loc. |
| BUILD      | ""                       | encoded string like 'army Berlin' |                 |
| RETREAT    | initial loc. of the unit | target loc                        |                 |
| DESTROY    | initial loc. of the unit |                                   |                 |

### What the map looks like ! 

<img src="img/map.png" width="900">

# Discovering the Dataset

Now that all of this is well-defined, let's see what we can achieve in the code. As it can be quite hard to see how to do this, let's break this down and look at one game.

In [5]:
# extract one game
# 37317
# 120600
game = all_games.query("id == 37317")
game_id = game.iloc[0,0]
game

,id,num_turns,num_players
0,37317,166,7


In [6]:
# for this game, extract turns, orders and units
turns = all_turns.query("game_id == {}".format(game_id))
orders = all_orders.query("game_id == {}".format(game_id))
units = all_units.query("game_id == {}".format(game_id))
orders.head()

,game_id,unit_id,unit_order,location,target,target_dest,success,reason,turn_num
11451415,37317,0,MOVE,Edinburgh,North Sea,None,1,None,1
11451416,37317,1,MOVE,Liverpool,Belgium,None,0,Illegal order replaced with Hold order,1
11451417,37317,2,HOLD,London,None,None,1,None,1
11451418,37317,3,MOVE,Marseilles,Spain,None,1,None,1
11451419,37317,4,MOVE,Paris,Burgundy,None,0,Bounced,1


## Can we find **acts of friendships** ?

It's firstly defined by a support. However it is not enough: a player could support himself (and that's not a friendship). So we must also look at the **last previous orders** asking to **MOVE** the unit towards the support's **target** destination. This will link to a 'unit_id' (the one that followed this order) and therefore giving access to the country who made the call.

In [7]:
# first we must look at the supports that happened in this game.
supports = orders.unit_order == "SUPPORT"
orders_w_supports = orders[supports]
orders_w_supports.sample(3)

,game_id,unit_id,unit_order,location,target,target_dest,success,reason,turn_num
11452686,37317,38,SUPPORT,Vienna,Galicia,Galicia,1,None,28
11452214,37317,33,SUPPORT,Galicia,Budapest,Rumania,0,Dislodged by a rum - gal,18
11452146,37317,18,SUPPORT,Rumania,Galicia,Galicia,0,Supported unit has failed,16


In [8]:
# we want to find the countries of the supported units
# let's take one and see what we can do
support_order = orders_w_supports.iloc[-4]
# support_order = orders_w_supports.head(3).tail(1)
support_order

game_id                             37317
unit_id                                12
unit_order                        SUPPORT
location                          Galicia
target                             Vienna
target_dest                       Bohemia
success                                 0
reason         Support cut by a war - gal
turn_num                               33
Name: 11452914, dtype: object

In [9]:
# Example: there is a support from 'Vienna' to 'Bohemia' 
# we know that in one of the previous orders, someone made a move with destination 'Vienna'
target = support_order.target#.values[0]
turn_number = support_order.turn_num#.values[0]
move_order = orders.query("unit_order == 'MOVE' & target == '{}' & turn_num < {}".format(target, turn_number)).tail(1)
move_order

,game_id,unit_id,unit_order,location,target,target_dest,success,reason,turn_num
11451517,37317,20,MOVE,Galicia,Vienna,None,0,Bounced,3


In [10]:
unit_id = move_order.unit_id.values[0]
move_unit = units.query("unit_id == {}".format(unit_id))
move_unit

,game_id,country,type,start_turn,end_turn,unit_id
1035275,37317,R,A,0,45,20


We see that rusian was the country who had moved it's army there the last time before a support happened. Hence, 'Russia is the supported Country'.

In [11]:
# Let's look at the country who did the support
unit_id = support_order.unit_id# .values[0]
support_unit = units.query("unit_id == {}".format(unit_id))
support_unit

,game_id,country,type,start_turn,end_turn,unit_id
1035256,37317,A,A,0,110,12


As it turns out this **is** an act of friendship: Russia was supported by Austria, when it moved from Vienna to Bohemia, by Austrian soldiers who were in Galicia. As we can see on the map, this is perfectly coherent with the geographical position of provinces.

## A little data pre-processing won't hurt us

Now is when we realize it would be nice to have an extra information about orders. This can be seen as a pre-processing step, since it will be computed only once. 

First thing that we want to have is to know **what country passed which order**.

As we will see later on, we will also need **the year and the season at which each order was passed**, in order to find friendships with more precisions than if we use only 'year' as indicator of when an order happens. Here is how we can encode this: 
- the spring of a year is writen as the year, for instance 'Spring 1904' = 1904
- the fall of the same year is written as the year + 0.5, for instance 'Fall 1904' = 1904.5

Also, there is sometimes a problem with the data. Order with id == 89268 for instance has the 'units' that are duplicated. Possible way to solve this is to run `units.drop_duplicates()`. It's not the only possible problem with the 'unit' array, indeed sometimes there are duplicates that are 'almost exactly the same, but not exactly. The end turn would be sligtly different. The solution to this problem is simply to expect an error and if one is catched, to skip this game.

In [12]:
try: 
    # 1. compute the country who emitted each order 
    countries = orders.unit_id.apply(lambda x: units.query("unit_id == {}".format(x)).country.item())
    # compute the 'encoded year' at which each order was passed
    turns["season_encoded"] = turns["year"] + 0.5 * (turns["season"] == "Fall")
    years = orders.turn_num.apply(lambda x: turns.query("turn_num == {}".format(x)).season_encoded.item())
    # update the orders
    orders["country"] = countries
    orders["year"] = years
except ValueError:
    print("Data is corrupted and execution was aborted")

Finally, a last step we will need is more **information about the 'target country'**. When an order is passed, there is often another country located where the order's target is. This information is really important to us. Let's find it here.

In [13]:
def get_target_country(order, orders):
    """
    Given an order MOVE or SUPPORT, it returns the country that controls the targeted province.
    The function looks at 'the last successful move towards the targetted province'.
    If none is found, returns None.
    If order is not MOVE or SUPPORT, it returns empty string.
    """
    if order.unit_order not in ["MOVE", "SUPPORT"]: return ""
    # get the last move orders to this province
    q = "(unit_order in ('MOVE' ,'RETREAT', 'HOLD')) & success == 1 & target == '{}' & turn_num < {}".format(order.target, order.turn_num)
    last_move_orders = orders.query(q)
    if len(last_move_orders):
        # extract the order
        last_move_order = last_move_orders.iloc[-1]
        return last_move_order.country
    else:
        # this is when there never was a successful move toward this 
        return "None"

In [14]:
# get target country
targets = orders.apply(lambda o: get_target_country(o, orders), axis = 1)
orders["target_country"] = targets
orders.sample(5)

,game_id,unit_id,unit_order,location,target,target_dest,success,reason,turn_num,country,year,target_country
11451907,37317,31,HOLD,Ankara,None,None,1,None,11,T,1903.0,
11451926,37317,19,SUPPORT,Sweden,St. Petersburg,Norway,1,None,11,R,1903.0,None
11451774,37317,11,HOLD,Belgium,None,None,1,None,8,G,1902.5,
11452693,37317,28,MOVE,Bohemia,Silesia,None,0,Bounced,28,A,1906.5,R
11452306,37317,23,SUPPORT,London,English Channel,North Sea,1,None,21,F,1905.0,F


## Acts of **friendships** and Acts of **hostility**

Now that we have a good dataframe, let's start the processing.

### Finding **acts of friendship**

This function is just a rewriting of all the code which was presented above.

Note that we don't add the query criteria 'success == 1' because it's not the result which defines the intention of the act. You can plan to attack someone and fail, you will still have performed a hostile action toward the other player.

In [15]:
def is_order_act_of_friendship(support_order):
    # only interested in SUPPORT orders to other countries
    if support_order.unit_order != "SUPPORT": return False
    if support_order.target_country == "None": return False
    return support_order.country != support_order.target_country

In [16]:
# so we may add this to the orders dataframe
acts_of_friendships = orders.apply(is_order_act_of_friendship, axis = 1)
acts_of_friendships.value_counts()

False    461
True      21
dtype: int64

Good news : it's working. As we see, there is another column 'acts_friendships' which tell if this order was an act of friendships-

### Finding **acts of hostility**

The code is quite the same, just the logic is a little tweaked. 

We are looking at all **orders** with **unit_order** that is **MOVE** (*this is how an attack starts*), where the **target** is a province with a unit of another player. Again, the tricky part is to see if there is another player's unit located at this province. We must look at all **previous orders** with the same **target** done by a unit from **another country**.

In [17]:
def is_move_act_of_hostility(move_order):
    if move_order.unit_order != "MOVE": return False
    if move_order.target_country == "None": return False
    return move_order.country != move_order.target_country

In [18]:
acts_of_hostility = orders.apply(is_move_act_of_hostility, axis = 1)
acts_of_hostility.value_counts()

False    410
True      72
dtype: int64

## Who is friend with who

Now that we have classified our orders, let's find friends within our game. Coming back to our definition:

> Friendship is a relationship between two players spanning over 3 seasons containing at least 2 consecutives and reciprocated acts of friendships.

The difficulty comes from the identification of the country who received the support. Let's add another function to solve this problem which we will be able to call for both the acts of friendships and the acts of hostility. In both case, it's the **target province** that matters.

In [19]:
friendly_orders = orders[acts_of_friendships]
hostile_orders = orders[acts_of_hostility]

For each order in the above set , assuming the 'supporting country' is X and the 'supported country' is Y, we must look if there exist in the last 2 years: 
- at least 1 previous order with 'X supports Y'
- at least 2 previous orders with 'Y supports X'

Those acts of frienships are part of a **friendships**.

Let's create a table which will recap all friendships and potential betrayal, and then let's fill it up with the data already extracted. 

In [20]:
def get_countries_code(x,y):
    """Given two countries, it returns another string being their letters sorted alphabetically.
    This string is the 'encoded friendship name'."""
    return ''.join(sorted(x + y))

In [21]:
def analyse_friendships(df):
    """Given the friendship dataframe, it will extract insights out of it"""
    # get columns where something happened
    cols = [col for col in df.columns if np.count_nonzero(friendships[col] != 0)]
    if len(cols):
        print("- friendships:", cols)
        for c in cols: 
            tmp = df.loc[:,c]
            pos = tmp[tmp != 0].values
            # print(pos)
            if type(pos[-1]) is str: 
                print("    * length of friendship ",c," is ", len(pos) - 1)
                print("        ->",c," ends betrayed by ",pos[-1],"") 
            else: 
                print("    * length of friendship: ", len(pos))

    else:
        print("- No friendships was found")

Our first attempt to find friendships and betrayals led us to write those 2 functions. They work 95% of the times, but not always. The often identify friendships that are too short, for instance.

In [22]:
countries = ['A', 'E', 'F', 'G', 'I', 'R', 'T']
pairs = [x+y for x in countries for y in countries if y > x]
years = np.arange(1901, max(friendly_orders.year.max(), hostile_orders.year.max()) + 0.5, 0.5)
# create a dataframe each possible friendships and each years
friendships = pd.DataFrame(columns=pairs, index = years, dtype = np.int8).fillna(0)

In [23]:
def find_friendships(friendships, friendly_orders, hostile_orders, length_year = 1.5):
    """Will fill the 'friendships' dataframe by adding '1's to signify that a pair of players engaged in a 
    reciprocal friendships
    
    Careful, the function doesn't return anything - instead it changes the 'friendship' dataframe
    
    Parameters
    friendships (pd.DataFrame): df with rows as 'encoded seasons' and columns as 'encoded friendship names'.
    friendly_orders (pd.DataFrame): frame of all order with an 'act of friendship'
    length_year (int): minimum amount of time that must pass between the first AOF and the last one.
    
    Discussion
    For each order with an AOF, it will query all other ones that 
    - happened before
    - involved exactly the same 2 countries
    Then it makes sure that amongs all those orders
    - each player originated at least 2 of them
    - that the time spent between the first one and the last one was longer than `length_year` 
    Those orders are the ones defining a friendships.
    """
    # for each friendly order, finds those that define a friendship
    for i, o in friendly_orders.iterrows():
        x, y, year = o.country, o.target_country,  o.year
        # this line doesn't remove frienships because we are only looking are reciprocal ones
        if y > x: continue
        # make a query over others friendly orders from the past (including this one !)
        q = "year <= @year & ( (country == @x & target_country == @y) | (country == @y & target_country == @x) )"
        query = friendly_orders.query(q)
        # extract number of helps from 2 countries
        n_x = np.count_nonzero(query.country == x)
        n_y = len(query) - n_x
        y_min = query.year.min()
        y_max = query.year.max()
        print("output of query: ",get_countries_code(x,y), y_min, y_max)
        if n_x >= 2 and n_y >= 2 and (y_max - y_min) >= length_year:
            # they meet all criteria about 'being nice to each other'
            code = get_countries_code(x,y)
            friendships.loc[query.year.min():o.year, code] = 1
            # we just want to make sure they are not 'being not nice' as well ! 
            # each time we find an AOH, we must break the friendship at this year
            q = "year < @y_max & year > @y_min & country in (@x, @y) & target_country in (@x, @y)"
            query = hostile_orders.query(q)
            for i, hostile_order in query.iterrows(): 
                print("   hostile act detected ", code, hostile_order.year, hostile_order.unit_id)
                friendships.loc[hostile_order.year, code] = 0            
    print(friendships[[c for c in friendships.columns if np.count_nonzero(friendships[c])]])

In [24]:
def find_betrayals(friendships, hostile_orders, N_hostile_min = 2):
    """Will fill the 'friendships' dataframe by removing '1' when a friendship was broken and if a 
    betrayal is detect, it will add its country letter to the frame. 
    
    Careful, the function doesn't return anything - instead it changes the 'friendship dataframe
    
    Parameters
    friendships (pd.DataFrame): df with rows as 'encoded seasons' and columns as 'encoded friendship names'.
    hostile_orders (pd.DataFrame): frame of all order with an 'act of hostility'
    N_hostile_min (int): minimum number of hostile action to consider this event as a betrayal
    
    Discussion
    For each order, if the players were engaged in a friendship, it will first destroy the friendship and 
    then verifiy that the event is considered as a betrayal according to our definition.
    """
    for i, o in hostile_orders.iterrows():
        x = o.country
        y = o.target_country
        code = get_countries_code(x,y)
        if friendships.loc[o.year, code]:
            # 1. it breaks the friendship
            friendships.loc[o.year+0.5:, code] = 0 
            # 2. did a betrayal happened ? 
            # we must just verify that another hostile action happened after this
            q = "year >= {}  & ((country == @x & target_country == @y) | (country == @y & target_country == @x) )".format(o.year, x, y)
            query = hostile_orders.query(q)
            if len(query) >= N_hostile_min: 
                # print("   - betrayal happened for ", code)
                friendships.loc[o.year, code] = x 

In [25]:
# let's test this function
years = np.arange(1901, max(friendly_orders.year.max(), hostile_orders.year.max()) + 0.5, 0.5)
friendships = pd.DataFrame(columns=pairs, index = years, dtype = np.int8).fillna(0)
find_friendships(friendships, friendly_orders, hostile_orders)
find_betrayals(friendships, hostile_orders)
analyse_friendships(friendships)

output of query:  AI 1904.5 1904.5
output of query:  AI 1904.5 1905.0
output of query:  RT 1905.0 1905.0
output of query:  GR 1905.0 1905.0
output of query:  GI 1905.5 1905.5
output of query:  FR 1905.5 1905.5
output of query:  GI 1905.5 1906.0
output of query:  GR 1905.0 1906.0
output of query:  RT 1905.0 1906.5
output of query:  FR 1905.5 1906.5
output of query:  GI 1905.5 1907.0
output of query:  GR 1905.0 1907.0
output of query:  AI 1904.5 1907.5
   hostile act detected  AI 1905.0 24
output of query:  GR 1905.0 1907.5
         AI
1901.0  0.0
1901.5  0.0
1902.0  0.0
1902.5  0.0
1903.0  0.0
1903.5  0.0
1904.0  0.0
1904.5  1.0
1905.0  0.0
1905.5  1.0
1906.0  1.0
1906.5  1.0
1907.0  1.0
1907.5  1.0
- friendships: ['AI']
    * length of friendship:  6


Even if those function are not definitive, they helped us a lot to identify a better code, which is explained now.

### Finding Friendships Algorithm

The final algorithm to find friendships and betrayals is a rather complex, so here is a high-level explanation of what it does.

*Abreviations used recurrently in the code*
- AoF = "Act of friendship"
- AoH = "Act of hostility"

The main idea of the algorithm is to consider series of AoFs that we calls '**bags of AoF**'. Those bags, with what separates them, is what to be analysed. Let's define 2 criteria, internal and external. 

a. **Internal criteria** that a bag of AOFs must verify (*internal* because it only depends on the AoFs only)
- more than 2 acts emitted by the 2 countries
- the overall time elapsed is longer than 1.5 years

b. **External criteria** that a bag of AoFs must verify (*external* because the criteria depend on the AoHs that happened at the same time)
- no AoH in the same time range


It goes like this:

**For each pair of countries** (*their friendship is represented by a column of the 'friendships' dataframe*): 
1. Find all AOFs that happened between them. Consider them as a bag of AoFs. 
2. Verify that this 'bag of acts of friendships' (BAOF) verifies the *internal criteria* mentioned above. If so, fill between the first season and the last season with '1's. Else, make sure '0' are placed at the season of interest. 
3. Find acts of hostility that happened between the same time range as the bag of words. Split the bag of words (*actually, it's the seasons array that is split*) for every act of hostility so that we obtain 
4. For each bag of AoF (hence, un-interupted by hostility, so external validity is verified !) verify again internal validity. If not verified, replace '1's with zeros.
5. Find betrayal, as the event:  "breaking a bags of AoFs with more than 1 hostile action"


Let's note that since the step 3 doesn't need to be repeated, it is not a recursive aglorithm. 

In order to implement this algorithm, we need a 'splitting' function which also is a little tricky. There are several ways to do this splitting properly (recursive spliting would be one way) but we will use a vectorized approach to do so. We want to split a list of 'ones' divided by zeros into a list of 'bags' of ones. Let's just show how one can do that here

In [26]:
# illustration of the splitting algorithm 
ones = np.array([1., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1.])
delta = ones[1:] - ones[:-1]
idcs = np.where(delta != 0)[0]+1
bags = np.split(ones, idcs)
bags

[array([1.]),
 array([0., 0.]),
 array([1., 1., 1.]),
 array([0., 0., 0., 0., 0., 0.]),
 array([1.])]

As we can see, it works well ! Let's just write a few functions for those tasks, and then we can implement the overall algorithm safely.

In [27]:
length_year = 1.5
countries = ['A', 'E', 'F', 'G', 'I', 'R', 'T']
pairs_tuple = [(x,y) for x in countries for y in countries if y > x]

def are_AOFs_friendship(AOFs, x, y, length_year = 1.0, n_xy_min = 2, n_min = 4):
    """Internal validity criteria:
    Given a dataframe of 'AoF's, and the 2 countries of interest, 
    it returns the validity of this series of friendly actions. 
    """
    if not len(AOFs): return False, None, None
    # extract number of helps from both countries
    n_x = np.count_nonzero(AOFs.country == x)
    n_y = len(AOFs) - n_x
    # extract information about the year
    y_min = AOFs.year.min()
    y_max = AOFs.year.max()
    # print(get_countries_code(x,y), y_min, y_max)
    return (n_x >= n_xy_min and n_y >= n_xy_min and (n_x + n_y) >= n_min and (y_max - y_min) >= length_year), y_min, y_max

def get_bags_of_ones_idcs(ones):
    """Given an numpy array of ones, returns the indices where to split the array 
    so that all ones are together and all zeros together, in sub arrays, once split."""
    delta = ones[1:] - ones[:-1]
    idcs = np.where(delta != 0)[0]+1
    return idcs

Here is the infamous function.

In [28]:
def find_friendships_and_betrayals(friendships, friendly_orders, hostile_orders, N_hostile_min = 2, ):
    """Fills the 'friendships' dataframe by adding '1's to signify that a pair of players engaged in a 
    reciprocal friendship.
    
    Careful, the function doesn't return anything - instead it changes the 'friendships' dataframe
    
    Parameters
    friendships (pd.DataFrame): df with rows as 'encoded seasons' and columns as 'encoded friendship names'.
    friendly_orders (pd.DataFrame): frame of all orders with an 'act of friendship'
    length_year (int): minimum amount of time that must pass between the first AOF and the last one.
    N_hostile_min (int): min number of AoH so that a broken friendship is considered as a betrayal. 
    """
    for pair in pairs:
        # 1. make a query for all acts of support of this pair of countries    
        x, y = pair
        q = "country in (@x, @y) & target_country in (@x, @y)"
        query1 = friendly_orders.query(q)
        # 2. verify internal validity of AoFs
        is_valid, y_min, y_max = are_AOFs_friendship(AOFs = query1, x = x, y = y)
        if is_valid:
            # Fill with '1' everywhere
            code = get_countries_code(x,y)
            friendships.loc[y_min:y_max, code] = 1
            # Look for hostilities
            q = "year < @y_max & year > @y_min & country in (@x, @y) & target_country in (@x, @y)"
            query2 = hostile_orders.query(q)
            # Each time we find an AOH, break the friendship at this season
            for i, hostile_order in query2.iterrows(): 
                friendships.loc[hostile_order.year, code] = 0    
            # Verify again that each small bag of AoFs is valid or not
            # a. get the indices using the values 
            idcs = get_bags_of_ones_idcs(friendships.loc[y_min:y_max, code].values)
            # b. and make the split using the indices of the dataframe
            bags_of_years = np.split(friendships.loc[y_min:y_max, code].index.values, idcs)
            # c. loop inside even (not odd) bags of years (the other ones are those with betrayals)
            # we know the first one '0' will be containing a friendship, that's why we can do this
            for years in bags_of_years[::2]: 
                AoFs = query1.query("year in @years")
                # print(AoFs[["year","unit_order", "country", "target_country"]], "\n")
                is_valid, _, _ = are_AOFs_friendship(AOFs= AoFs, x = x, y = y, n_xy_min = 1, n_min = 3)
                if not is_valid:
                    # this friendship is cancelled
                    friendships.loc[years, code] = 0  
            # Final step: find betrayals inside odd bags of years
            for years in bags_of_years[1::2]:
                # first, verify it's coming after a friendship
                if friendships.loc[years[0]-0.5, code]:   
                    # get the hostile orders in the desired time interval 
                    AoHs = query2.query("year in @years")
                    if len(AoHs) >= N_hostile_min:
                        # betrayal happened here ! 
                        betrayal = AoHs.iloc[0]
                        friendships.loc[betrayal.year, code] = betrayal.country

In [29]:
# let's test this function
years = np.arange(1901, max(friendly_orders.year.max(), hostile_orders.year.max()) + 0.5, 0.5)
friendships = pd.DataFrame(columns=pairs, index = years, dtype = np.int8).fillna(0)
find_friendships_and_betrayals(friendships, friendly_orders, hostile_orders)
analyse_friendships(friendships)

- friendships: ['AI']
    * length of friendship:  5


**Note for debugging**: *it is possible to nicely vizualize in a terminal the 'frienships' dataframe and the orders with the following commands**
- to see acts of friendships: `friendly_orders[["year", "country", "target_country"]]`
- same by filtering for 2 countries: `friendly_orders[["year", "country", "target_country"]].query("country in ('E', 'G')")`
- or even the more complete `friendly_orders[["year", "country", "target_country"]].query("country in ('E', 'G') and target_country in ('E', 'G')")` 
- to see friendships that matters: `friendships[[c for c in friendships.columns if np.count_nonzero(friendships[c])]]`

In [30]:
friendships[[c for c in friendships.columns if np.count_nonzero(friendships[c])]]

,AI
1901.0,0.0
1901.5,0.0
1902.0,0.0
1902.5,0.0
1903.0,0.0
1903.5,0.0
1904.0,0.0
1904.5,0.0
1905.0,0.0
1905.5,1.0


We see here what happened between Austria and Italy: in the Spring of 1905, Italy attacked Austria... This is why the friendship ended. However, it is not count as a betrayal, because our definition asks for 2 hostile actions and in this case, there was only one. 

# Scaling to several games

Until now, we have been only looking at a single game, and we must start to scale the analysis to a broader range. Let's see what we can do. The first thing to do is to make the process a little more functional by defining methods for each step of the process.

In [31]:
def get_dataframes(game_id):
    orders = all_orders.query("game_id == {}".format(game_id))
    turns = all_turns.query("game_id == {}".format(game_id))
    units = all_units.query("game_id == {}".format(game_id))
    return orders, turns, units

In [32]:
def get_empty_friendships(max_year):
    countries = ['A', 'E', 'F', 'G', 'I', 'R', 'T']
    pairs = [x+y for x in countries for y in countries if y > x]
    years = np.arange(1901, max_year + 0.5, 0.5)
    # create a dataframe each possible friendships and each years
    return pd.DataFrame(columns=pairs, index = years, dtype = np.int8).fillna(0)

In [33]:
def process_orders(orders, turns, units):
    try:
        # 1. compute the country who emitted each order 
        countries = orders.unit_id.apply(lambda x: units[["unit_id", "country"]].query("unit_id == {}".format(x)).country.item())
        orders["country"] = countries
        # 2. compute the 'encoded year' at which each order was passed
        turns["season_encoded"] = turns["year"] + 0.5 * (turns["season"] == "Fall")
        years = orders.turn_num.apply(lambda x: turns.query("turn_num == {}".format(x)).season_encoded.item())
        orders["year"] = years
        # 3. get target countries
        targets = orders.apply(lambda o: get_target_country(o, orders), axis = 1)
        orders["target_country"] = targets
        return True
    except ValueError:
        print("Data is corrupted and execution was aborted")
        return False

In [60]:
def analyse_game(game, verbose = False):
    # get dataframes and process them
    orders, turns, units = get_dataframes(game_id = game.id.item())
    has_succeeded = process_orders(orders, turns, units)
    if not has_succeeded: return None
    
    # find acts of friendships / hostility
    acts_of_friendships = orders.apply(is_order_act_of_friendship, axis = 1)
    acts_of_hostility = orders.apply(is_move_act_of_hostility, axis = 1)
    friendly_orders = orders[acts_of_friendships]
    hostile_orders = orders[acts_of_hostility]
    if verbose:
        print("- {} acts of friendship and {} acts of hostility over {} orders ".format(len(friendly_orders), len(hostile_orders), len(orders)))
    
    # construct friendship matrix
    if len(friendly_orders) and len(hostile_orders):
        friendships = get_empty_friendships(max_year = max(friendly_orders.year.max(), hostile_orders.year.max()))
        find_friendships_and_betrayals(friendships, friendly_orders, hostile_orders)
        return friendships, friendly_orders, hostile_orders
    else: 
        return None

Now that we have all our functions, we can verify that we obtain the same results as before.

In [63]:
game = all_games.query("id == 78236")
friendships, _, _ = analyse_game(game)
if friendships is not None:
    analyse_friendships(friendships)
else:
    print("No acts of friendships or no acts of hostility")

- friendships: ['AT', 'EG']
    * length of friendship:  7
    * length of friendship  EG  is  3
        -> EG  ends betrayed by  E 


We may now try to use those results over more games ! The first thing that one observes is that according to our definition, not all games contain friendships, and most of the friendships ends - at some point - in a betrayal. This make sense with the 'mindset of the game'. Here is a quote of the creator of the game. No wonders we observe a lot of betrayals.

> "Luck plays no part in Diplomacy. Cunning and cleverness honesty and perfectly-timed betrayal are the tools needed to outwit your fellow players. The most skillful negotiator will climb to victory over the backs of both enemies and friends. Who do you trust?"

In [36]:
# let's run this function many times
games_sample = all_games.query("num_turns >= 60").sample(1)
for i, game in games_sample.iterrows(): 
    print("\n*** Game {} ***".format(game.id.item()))
    friendships, _, _ = analyse_game(game)
    if friendships is not None:
        analyse_friendships(friendships)


*** Game 75248 ***
- friendships: ['EG']
    * length of friendship:  12


We notice that for many games, there are very few acts of friendships. For instance the game '91 843' has 1002 orders that were passed and only '2' of them were considered as acts of friendships. We guess that there are two types of games: those with many alliances and betrayals, and those with more personal games and players not coordinating a lot. We are only interested into the second types, so we will filter a little bit those games the following way.


# Dataset Generation 

As the dataset is really heavy, we decided to generate smaller dataset only containing the data that we care about.

## Overview of what friendship looks like

The next block of code generates a portion of the dataset as numpy object so that we can later analyse it better.

In [37]:
n_games = 0
verbose = False

games = all_games.query("num_turns >= 50")

game_ids = []
all_friendships = []

s = time.time()
while n_games:
    game = games.query("id not in @game_ids").sample(1)
    game_id = game.id.item()
    friendships, _, _ = analyse_game(game, verbose = verbose)
    if friendships is not None:
        cols = [col for col in friendships.columns if np.count_nonzero(friendships[col] != 0)]
        if len(cols):
            n_games -= 1
            game_ids.append(game_id)
            all_friendships.append(friendships.values)
            if verbose:
                print("Valid GAME was found: ", game_id)
                analyse_friendships(friendships)
                print("")
e = time.time()

print("elapsed: ", e-s)
np.save("data/subset2/games_id.npy", np.array(game_ids))
np.save("data/subset2/friendships.npy", np.array(all_friendships, dtype=object))

elapsed:  0.00016546249389648438


## Features generation for machine learning analysis

Let's generate some features. We want to have 250 betrayals and 250 good friendships. Then, for each of those games, we will extract the seasons of the friendships and get some features for each of those seasons.

In [328]:
def get_xy(code, seasons): 
    # find x and y
    xy = list(code)
    if type(seasons[-1]) == str: 
        x = seasons[-1]
    else:
        x = xy[np.random.randint(2)]
    xy.remove(x)
    y = xy[0]
    return x,y


columns = ["game_id", "has_betrayal", "length", "outcome" ,"n_aoh_x", "n_aoh_y", "n_aos_x", "n_aos_y", "supports_xy" , "supports_yx"]


def get_features(game_id, has_betrayal, friendship, friendly_orders, hostile_orders):
    """
    Given the friendships' column representing 1 friendship, 
    it will create all the feature vectors for each seasons, and will return them all together 
    as a Pandas dataframe.
    
    Parameters
    friendship (pd.Serie): ...
    friendly_orders: all friendly orders for this game
    hostile_orders: all hostile orders for this game
    
    Notations
    X = 'betrayer'
    Y = 'betrayed' = 'victim'
    
    Features of each seasons
    length: length of friendship at this season
    n_aoh_x: number of 'acts of hostility' emmited by player x this seasons
    """
    # n = len(friendship[friendship != 0])
    n = len(friendship[friendship == 1])
    features = pd.DataFrame(data=np.zeros(shape=(n, len(columns))),columns = columns)
        
    
    # find betrayals inside this friendship
    betrayals = [type(s) == str for s in friendship]
    code = friendship.name
    # some very rare games have more than 1 betrayal: 90346
    # hence we split the friendshiè into its sub-component
    splitted = np.split(friendship.values, np.where(betrayals)[0]+1)
    print(splitted)
    

    # i: iterator over rows inside the features dataframe
    # j: iterator insiide the splitted array we are using
    # s: iterator ove which splitted array we are using
    i,j,s = 0,0,0
    for year, season in friendship.iteritems():
        if not season: continue
            
        if type(season) == str: 
            # the betrayals happens --> let's pass to the next splited
            features.loc[i-1,"outcome"] = 1
            s += 1
            j = 0
        else:
            # it means this is a season where: 
            # - the 2 playes are engaged in a friendship
            # - there is no betrayal (yet...)
            x,y = get_xy(code, splitted[s])
            # 1. Easy features
            features.loc[i,"game_id"] = game_id
            features.loc[i,"has_betrayal"] = has_betrayal
            features.loc[i,"length"] = j+1
            features.loc[i,"outcome"] = 0
            # 2. number of acts of hostility
            # a. n_aoh_x
            h_query = hostile_orders.query("country == @x & year == @year")
            features.loc[i,"n_aoh_x"] = len(h_query)
            # b. n_aoh_y
            h_query = hostile_orders.query("country == @y & year == @year")
            features.loc[i,"n_aoh_y"] = len(h_query)
            # 3. number of AoS emitted towars other players 
            # a. n_aos_x
            f_query = friendly_orders.query("country == @x & target_country != @y & year == @year")
            features.loc[i,"n_aos_x"] = len(f_query)
            # a. n_aos_y
            f_query = friendly_orders.query("country == @y & target_country != @x & year == @year")
            features.loc[i,"n_aos_y"] = len(f_query)
            # 4. Average number of mutual supports for last 3 seasons
            supports_yx = friendly_orders.query("country == @y & target_country == @x & year <= @year & year > (@year-1.5)"
                                               ).groupby("year").apply(len)
            if len(supports_yx): 
                features.loc[i,"supports_yx"] = supports_yx.mean()
            supports_xy = friendly_orders.query("country == @x & target_country == @y & year <= @year & year > (@year-1.5)"
                                               ).groupby("year").apply(len)
            if len(supports_xy):
                features.loc[i,"supports_xy"] = supports_xy.mean()
            # last. increment steppers
            i += 1 
            j += 1
            
    return features    

In [329]:
# try this fonction with one game in particular 
# IDs with several things: 90346, 86785
game = all_games.query("id == 86785")
game_id = game.id.item()
friendships, friendly_orders, hostile_orders = analyse_game(game, verbose = True)
cols = [col for col in friendships.columns if np.count_nonzero(friendships[col] != 0)]
c = cols[0]
friendship = friendships[c]

- 41 acts of friendship and 239 acts of hostility over 1077 orders 


In [331]:
### Code to generate machine learning dataset

N = 1
verbose = True

visited_ids = []
n_betrayal, n_no_betrayal = N,N
features = pd.DataFrame(data=np.zeros(shape=(0, len(columns))),columns = columns)

s = time.time()
games = all_games.query("num_turns >= 50")
while n_betrayal or n_no_betrayal:
    # get one game
    game = games.query("id not in @visited_ids").sample(1)
    game_id = game.id.item()
    visited_ids.append(game_id)
    
    # perform our analysis of the game, and test for skip condition
    friendships, friendly_orders, hostile_orders = analyse_game(game, verbose = verbose)
    if friendships is None: continue
        
    # extract columns that matters, and test for skip condition
    cols = [col for col in friendships.columns if np.count_nonzero(friendships[col] != 0)]
    if not len(cols): continue
        
    # for each friendship:
    for c in cols:
        friendship = friendships[c]
        # 1. do the counts of betrayals and no betrayals
        has_betrayal = False
        if type(friendship[friendship != 0].iloc[-1]) == str:
            # there is a betrayal
            has_betrayal = True
            if n_betrayal: 
                print("betrayal detected")
                n_betrayal -= 1
                # todo: find features
        elif n_no_betrayal:
            # there wasn't a betrayal
            print("no betrayal detected")
            n_no_betrayal -= 1
            
        # 2. get the features
        game_features = get_features(game_id, has_betrayal, friendship, friendly_orders, hostile_orders)
        features = pd.concat([features, game_features])
            

    if verbose:
        print("Valid GAME was found: ", game_id)
        analyse_friendships(friendships)
        print("")
e = time.time()

print("elapsed: ", e-s)
features.to_pickle("data/features1/features.pkl")
features

- 19 acts of friendship and 247 acts of hostility over 1348 orders 
no betrayal detected
[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0.])]
Valid GAME was found:  67516
- friendships: ['GR']
    * length of friendship:  3

- 59 acts of friendship and 202 acts of hostility over 887 orders 
betrayal detected
[array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       1.0, 1.0, 1.0, 'E'], dtype=object), array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], dtype=object)]
Valid GAME was found:  55116
- friendships: ['EF']
    * length of friendship  EF  is  3
        -> EF  ends betrayed by  E 

elapsed:  10.601808547973633
